Before running the code, several dependant package need to be installed, skip if it already exists.

In [2]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pdfminer.six

     |████████████████████████████████| 5.6MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install python-Levenshtein

Firstly, we make connection with database with pymysql api.

In [ ]:
import pymysql
#input basic info about database
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
#cursor is the current point we focus on, which could execute sql command.
cursor=conn.cursor()
lst=[]
#Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
cursor.execute("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    lst.append(list(r))
lst

After excution we obtained a 2-dimensional matrix with each row represents a paper. Next, we constract a url based on arXiv id.

In [7]:
name=lst[7][4]
url="https://arxiv.org/pdf/"+str(name)+".pdf"
print(url)


https://arxiv.org/pdf/1909.01807.pdf


By following command we could access pdf by url and convert pdf 2 txt file and store at local.

In [16]:
#https://cloud.tencent.com/developer/article/1395339
import urllib
import pdfminer
from io import StringIO
from io import BytesIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import *
from pdfminer.converter import PDFPageAggregator
from urllib.request import Request
from urllib.request import urlopen
# 定义解析函数

def OnlinePdfToTxt(dataIo,new_path):
    # 创建一个文档分析器
    parser = PDFParser(dataIo)
    # 创建一个PDF文档对象存储文档结构
    document = PDFDocument(parser)
    # 判断文件是否允许文本提取
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        # 创建一个PDF资源管理器对象来存储资源
        resmag =PDFResourceManager()
        # 设定参数进行分析
        laparams=LAParams()
        # 创建一个PDF设备对象
        # device=PDFDevice(resmag )
        device=PDFPageAggregator(resmag ,laparams=laparams)
        # 创建一个PDF解释器对象
        interpreter=PDFPageInterpreter(resmag ,device)
        # 处理每一页
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            # 接受该页面的LTPage对象
            layout=device.get_result()
            for y in layout:
                try:
                    if(isinstance(y,LTTextBoxHorizontal)):
                        with open('%s'%(new_path),'a',encoding="utf-8") as f:
                            f.write(y.get_text()+'\n')
#                             print("读入成功！")
                except:
                    print("读入失败!")

# 获取文件的路径
#url = "https://arxiv.org/pdf/2004.11055.pdf"
html = urllib.request.urlopen(urllib.request.Request(url)).read()
dataIo = BytesIO(html)
OnlinePdfToTxt(dataIo,'txt/'+str(name)+'.txt')

ImportError: cannot import name 'uint_value' from 'pdfminer.pdftypes' (/Applications/anaconda3/lib/python3.7/site-packages/pdfminer/pdftypes.py)

From the generated txt file, first we try to extract the email address by using regular experssion
### TODO: Special format such as {aaa, bbb, ccc}@xxx.com


In [7]:
import re

#open the generated file as input stream and store in the buffer
def email_extraction(file_name):
    file = open('txt/'+str(file_name)+'.txt',encoding="ISO-8859-1")
    strings=file.read()
    matches = []
    emailRegex = re.compile(r'''(
        [a-zA-Z0-9._%+-]+      # username

        @                      # @ symbol

        [a-zA-Z0-9.-]+        # domain name

        (\.[a-zA-Z]{2,4}){1,2} # dot-something

        )''', re.VERBOSE)
    # using RE to match all the patterns in the txt
    for groups in emailRegex.findall(strings):
        matches.append(groups[0])
    # reduced the same entities
    list2 = list(set(matches))
    list_nums = len(list2)
    emailList=[]
    for line in range(list_nums):
        emailList.append(list2[line])
    return emailList

email_extraction(name)

NameError: name 'name' is not defined

Secondly we match the author name list with the email list by token matching method (Levenshtein)
### TODO: HOW can we access author list? 

In [ ]:
from Levenshtein import *
def author_email_matching(nameList, emailList):
    if len(emailList)==0:
        return
    name_email={}
    for name in nameList:
        highest=0
        index=0
        for i in range(len(emailList)):
            #using re to extract the first part of email
            pat = re.compile(''+'(.*?)'+'@', re.S)
            email = pat.findall(emailList[i])
            #print(jaro(name, str(email)))
            #using jaro to calculate the similarity between two strings
            if jaro(name, emailList[i])>highest:
                index=i
                highest=jaro(name, emailList[i])
        #set pair with the highest score
        name_email[name]=emailList[index]
        #If very sure, remove it from the list to reduce the uncertainty for other pairs
        if highest>=0.6:
            emailList.remove(emailList[index])
    return name_email
        

nameList=['Michael Stewart', 'Majigsuren Enkhsaikhan', 'Wei Liu']
author_email_matching(nameList, emailList)

After matching email, we move to the section of acknowledgement, which is not certain for every paper. Here we use named entity recognition to classify orgnization which sponsred the study. Here we used Spacy classifier.

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
#using Regular expression to locate the paragraph
def ORG_recognition(strings):
    keyStart = 'Acknowledg+'
    keyEnd = 'References'
    pat = re.compile(keyStart+'(.*?)'+keyEnd, re.S)
    result = pat.findall(strings)
    if len(result)<10:
        return
    #print(result)
    file.close()
    #format processing
    txt=''
    txt=txt.join(result)
    txt=txt.replace('\n', '').replace('\r', '')
    #print(txt)
    #using the pretrained model
    doc=nlp(txt)
    ORG_list=[]
    print([(X.text, X.label_) for X in doc.ents])
    for X in doc.ents:
        if X.label_=='ORG':
            ORG_list.append(X.text)
    return ORG_list